'''
A Reccurent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

MNIST 데이터를 가져옵니다.

In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


'''
To classify images using a reccurent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

본래 이미지는 기본적으로는 sequence라고 보기 그렇지만 일단 여기 예시에서는 28x28의 MNIST 데이터의 한줄 한줄을 sequence로 해석하도록 합니다. 즉 28 timestep의 시퀀스로 해석합니다.

아래는 파라미터와 placeholder를 준비하는 과정입니다.

* parameters
    * learning_rate, training_iters, batch_size, display_step
* network parameters
    * n_input: input data의 차원수 (한 time step에서의 데이터 차원수)
    * n_steps: time step수
    * n_hidden: LSTM에 사용할 hidden layer 갯수
    * n_classes: 최종 결과 class의 수
* placeholder 정의
    * x: [batch_size, n_steps, n_inputs] 차원으로 입력 정의
    * y: [batch_size, n_classes] 차원으로 출력 정의

In [2]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

RNN 생성을 위한 helper함수를 만든다.
* parameters
    * x: 입력
    * weights: weights (tf.Variable)
    * biases: weights (tf.Variable)
* input shape를 LSTM에 맞게 맞추는 부분
    * `tf.transpose()`를 통해 batch_size와 n_steps의 위치를 바꾼다. 즉, x=[n_steps, batch_size, n_input]이 된다.
    * `tf.reshape()`를 통해 [-1, n_input], 즉 x=[n_steps * batch_size, n_input]가 된다.
    * `tf.split()`를 통해 [n_steps * batch_size, n_input]를 0번째 차원을 기준으로 n_steps만큼 나누어 리스트를 만든다. 리스트 내의 하나의 tensor는 (batch_size, n_input)이 되고 이 tensor가 n_steps 만큼 있는 list가 반환된다.
* LSTM 구성 부분
    * `rnn.BasicLSTMCell()` 으로 lstm_cell 변수에 LSTM network에 사용할 기본 셀을 정의합니다. 이 때 차원수를 n_hidden만큼 두고, forget_bias를 아예 1.0으로 설정합니다.(기본값이라고 합니다.) google에 의하면 to reduce the scale of forgetting in the beginning of the training 용이라고 합니다.
    * `rnn.static_rnn()`함수를 통해 실제 LSTM network를 구성합니다. 입력으로 위에서 reshape등을 통해 생성한 x 를 받고 출력으로 출력 결과 outputs와 states 를 반환합니다.
* 최종 return은 LSTM 마지막(outputs[-1])에 대해 Ax+b 폼을 위한 형태로 return하여 줍니다.

In [3]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(x, n_steps, 0)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

위에서 만든 함수를 바탕으로 RNN을 구성합니다.
최종 결과는 multiclass classification이므로 softmax loss를 취하고 AdamOptimizer를 통해 최적화를 수행하도록 합니다.

In [4]:
pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

아래의 과정에서 실제 학습이 batch 단위로 일어나게 됩니다.
* CNN예시와 다른점은 tf graph에 넣기 전에 reshape를 통해 (batch_size, b_steps, n_input)으로 시퀀스 형태로 변환하여 넣었다는 점입니다.
* 약간 억지로 RNN 문제로 구성한 것이므로 성능이 아주 좋다고 하기엔 그렇지만 LSTM은 약간 만능툴(...) 같은 느낌이므로 어찌저찌 성능은 나옵니다.

In [5]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label})

Iter 1280, Minibatch Loss= 1.985561, Training Accuracy= 0.32812
Iter 2560, Minibatch Loss= 1.492967, Training Accuracy= 0.50781
Iter 3840, Minibatch Loss= 1.338582, Training Accuracy= 0.56250
Iter 5120, Minibatch Loss= 1.163514, Training Accuracy= 0.57812
Iter 6400, Minibatch Loss= 1.014229, Training Accuracy= 0.65625
Iter 7680, Minibatch Loss= 0.717111, Training Accuracy= 0.76562
Iter 8960, Minibatch Loss= 0.572510, Training Accuracy= 0.84375
Iter 10240, Minibatch Loss= 0.608282, Training Accuracy= 0.81250
Iter 11520, Minibatch Loss= 0.459709, Training Accuracy= 0.85156
Iter 12800, Minibatch Loss= 0.450477, Training Accuracy= 0.86719
Iter 14080, Minibatch Loss= 0.495952, Training Accuracy= 0.85938
Iter 15360, Minibatch Loss= 0.318509, Training Accuracy= 0.86719
Iter 16640, Minibatch Loss= 0.383801, Training Accuracy= 0.89062
Iter 17920, Minibatch Loss= 0.283810, Training Accuracy= 0.92969
Iter 19200, Minibatch Loss= 0.261616, Training Accuracy= 0.90625
Iter 20480, Minibatch Loss= 0.28